# Protein Data Analysis

In [1]:
import os
os.chdir("..")

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from helpers import load_config


In [3]:
configs = load_config(os.path.join("configs", "configs.yaml"))

In [4]:
study_path = configs["STORAGE_DIR_STUDY2"]

In [5]:
mutations_df = pd.read_csv(
    os.path.join(study_path, "data_mutations.txt"),
    sep="\t",
    header=0
)
mutations_df.head()

/var/folders/_0/qrjsyj4n0r951n63lt8qdzrc0000gn/T/ipykernel_22600/1016927232.py:1: DtypeWarning: Columns (4,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations_df = pd.read_csv(


,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,SYMBOL_SOURCE,TREMBL,TSL,UNIPARC,VARIANT_CLASS,all_effects,cDNA_position,n_depth,t_depth,Annotation_Status
0,DMBT1,1755.0,.,GRCh37,10,124399948,124399948,+,synonymous_variant,Silent,...,HGNC,B6V682_HUMAN,.,UPI000047021C,SNV,"DMBT1,synonymous_variant,p.%3D,ENST00000368956...",7054,115,131,SUCCESS
1,C10orf90,118611.0,.,GRCh37,10,128193330,128193330,+,missense_variant,Missense_Mutation,...,HGNC,"S4R3N7_HUMAN,Q5T025_HUMAN",.,UPI00001D808F,SNV,"C10orf90,missense_variant,p.Val100Met,ENST0000...",560,63,64,SUCCESS
2,WDFY4,57705.0,.,GRCh37,10,49997997,49997997,+,missense_variant,Missense_Mutation,...,HGNC,Q6PIM1_HUMAN,.,UPI000176ADB8,SNV,"WDFY4,missense_variant,p.Arg1345Trp,ENST000003...",4060,148,149,SUCCESS
3,PRKCQ,5588.0,.,GRCh37,10,6533674,6533674,+,missense_variant,Missense_Mutation,...,HGNC,.,.,UPI000012DF74,SNV,"PRKCQ,missense_variant,p.Gly254Val,ENST0000039...",861,197,228,SUCCESS
4,DYNC2H1,79659.0,.,GRCh37,11,103153768,103153768,+,missense_variant,Missense_Mutation,...,HGNC,.,.,UPI0000481AC7,SNV,"DYNC2H1,missense_variant,p.Pro3622Leu,ENST0000...",10865,204,199,SUCCESS


In [6]:
mutations_df.head().T

,0,1,2,3,4
Hugo_Symbol,DMBT1,C10orf90,WDFY4,PRKCQ,DYNC2H1
Entrez_Gene_Id,1755.0,118611.0,57705.0,5588.0,79659.0
Center,.,.,.,.,.
NCBI_Build,GRCh37,GRCh37,GRCh37,GRCh37,GRCh37
Chromosome,10,10,10,10,11
...,...,...,...,...,...
all_effects,"DMBT1,synonymous_variant,p.%3D,ENST00000368956...","C10orf90,missense_variant,p.Val100Met,ENST0000...","WDFY4,missense_variant,p.Arg1345Trp,ENST000003...","PRKCQ,missense_variant,p.Gly254Val,ENST0000039...","DYNC2H1,missense_variant,p.Pro3622Leu,ENST0000..."
cDNA_position,7054,560,4060,861,10865
n_depth,115,63,148,197,204
t_depth,131,64,149,228,199


In [7]:
mutations_df[mutations_df["Tumor_Sample_Barcode"] == "TCGA-AG-A020-01"].to_csv(
    os.path.join(study_path, "user_friendly_formats", "mutations_one_patient.csv"),
    index=False
)

In [8]:
df = mutations_df[["Hugo_Symbol", "HGVSp_Short"]].drop_duplicates()

In [9]:
# Check if the relationship Hugo_Symbol -> HGVSp_Short is one-to-one
hugo_to_hgvsp_unique = df.drop_duplicates(subset=['Hugo_Symbol', 'HGVSp_Short'])
hugo_to_hgvsp_grouped = hugo_to_hgvsp_unique.groupby('Hugo_Symbol')['HGVSp_Short'].nunique()
hugo_to_hgvsp_one_to_one = (hugo_to_hgvsp_grouped == 1).all()

# Check if the relationship HGVSp_Short -> Hugo_Symbol is one-to-one
hgvsp_to_hugo_unique = df.drop_duplicates(subset=['Hugo_Symbol', 'HGVSp_Short'])
hgvsp_to_hugo_grouped = hgvsp_to_hugo_unique.groupby('HGVSp_Short')['Hugo_Symbol'].nunique()
hgvsp_to_hugo_one_to_one = (hgvsp_to_hugo_grouped == 1).all()

if hugo_to_hgvsp_one_to_one and hgvsp_to_hugo_one_to_one:
    print("The relationship between Hugo_Symbol and HGVSp_Short is one-to-one.")
else:
    print("The relationship between Hugo_Symbol and HGVSp_Short is NOT one-to-one.")


The relationship between Hugo_Symbol and HGVSp_Short is NOT one-to-one.


In [10]:
df = mutations_df[["Tumor_Sample_Barcode", "HGVSp_Short"]].dropna().drop_duplicates()
df.shape

(276775, 2)

In [11]:
df.head()

,Tumor_Sample_Barcode,HGVSp_Short
0,TCGA-3L-AA1B-01,p.Y2316=
1,TCGA-3L-AA1B-01,p.V147M
2,TCGA-3L-AA1B-01,p.R1345W
3,TCGA-3L-AA1B-01,p.G254V
4,TCGA-3L-AA1B-01,p.P3615L


In [12]:
print("Number of unique proteins: ", len(df["HGVSp_Short"].drop_duplicates()))

Number of unique proteins:  124178


In [13]:
grouped = df.groupby('Tumor_Sample_Barcode')


mutations_df2 = grouped.agg(
    Protein_Count=('HGVSp_Short', 'count'),           # Define the name for the mutation count column
    Proteins=('HGVSp_Short', lambda x: ', '.join(x))  # Define the name for the aggregated mutation column
).reset_index()

mutations_df2.sort_values(by="Protein_Count", ascending=False)

,Tumor_Sample_Barcode,Protein_Count,Proteins
240,TCGA-AG-A002-01,12619,"p.F532=, p.F513=, p.R153Q, p.K58N, p.R584W, p...."
301,TCGA-CA-6717-01,11102,"p.S372L, p.A293=, p.I1009=, p.P1463=, p.S1039Y..."
480,TCGA-F5-6814-01,10592,"p.R766=, p.S239Y, p.*13*, p.A230T, p.K130=, p...."
150,TCGA-AA-A010-01,9129,"p.R267W, p.A30T, p.I388T, p.R387Q, p.E179K, p...."
278,TCGA-AZ-4315-01,7824,"p.A569V, p.V313=, p.K58N, p.P35S, p.D712G, p.*..."
...,...,...,...
201,TCGA-AG-3574-01,18,"p.R758Q, p.D127Y, p.*159*, p.A244=, p.E3795K, ..."
209,TCGA-AG-3594-01,17,"p.P148=, p.N174S, p.X475_splice, p.R937Q, p.P2..."
149,TCGA-AA-A00Z-01,17,"p.I1627=, p.R624H, p.P186L, p.*11*, p.Q164P, p..."
213,TCGA-AG-3605-01,15,"p.A103=, p.D28N, p.R142H, p.*137*, p.R229H, p...."


#### Keep only proteins that are present in atleast 30% of patients

In [14]:
total_barcodes = df['Tumor_Sample_Barcode'].nunique()
total_barcodes

528

In [15]:
threshold = 0.1 * total_barcodes
threshold

52.800000000000004

In [16]:
mutation_counts = df.groupby('HGVSp_Short')['Tumor_Sample_Barcode'].nunique()
mutation_counts.sort_values(ascending=False).head(100).to_csv(
    os.path.join("colorectal_adenocarcinoma", "top_100_proteins.csv"), 
    index=True
)

In [17]:
frequent_mutations = mutation_counts[mutation_counts >= threshold].index
frequent_mutations

Index(['p.*1*', 'p.*10*', 'p.*11*', 'p.*12*', 'p.*13*', 'p.*14*', 'p.*15*',
       'p.*16*', 'p.*17*', 'p.*18*', 'p.*19*', 'p.*2*', 'p.*20*', 'p.*21*',
       'p.*22*', 'p.*23*', 'p.*24*', 'p.*25*', 'p.*26*', 'p.*27*', 'p.*28*',
       'p.*29*', 'p.*3*', 'p.*30*', 'p.*31*', 'p.*32*', 'p.*33*', 'p.*34*',
       'p.*3454*', 'p.*35*', 'p.*36*', 'p.*38*', 'p.*39*', 'p.*4*', 'p.*41*',
       'p.*42*', 'p.*5*', 'p.*57*', 'p.*6*', 'p.*7*', 'p.*8*', 'p.*9*',
       'p.G12D', 'p.M1?'],
      dtype='object', name='HGVSp_Short')

In [18]:
filtered_df = df[df['HGVSp_Short'].isin(frequent_mutations)]
filtered_df

,Tumor_Sample_Barcode,HGVSp_Short
58,TCGA-3L-AA1B-01,p.G12D
98,TCGA-3L-AA1B-01,p.*18*
199,TCGA-4N-A93T-01,p.G12D
205,TCGA-4N-A93T-01,p.*19*
210,TCGA-4N-A93T-01,p.*20*
...,...,...
305139,TCGA-G5-6641-01,p.*11*
305177,TCGA-G5-6641-01,p.*26*
305227,TCGA-G5-6641-01,p.*21*
305228,TCGA-G5-6641-01,p.*13*


Since we are interested in just proteins and not the position data itself, lets filter out protein position records. 

In [19]:
filtered_df = df[~df['HGVSp_Short'].str.contains(r'\*')]
filtered_df

,Tumor_Sample_Barcode,HGVSp_Short
0,TCGA-3L-AA1B-01,p.Y2316=
1,TCGA-3L-AA1B-01,p.V147M
2,TCGA-3L-AA1B-01,p.R1345W
3,TCGA-3L-AA1B-01,p.G254V
4,TCGA-3L-AA1B-01,p.P3615L
...,...,...
305311,TCGA-AA-3864-01,p.G2168=
305313,TCGA-AA-3864-01,p.L28529=
305318,TCGA-AA-3864-01,p.N692=
305319,TCGA-AA-3864-01,p.N23=


In [20]:
mutation_counts = filtered_df.groupby('HGVSp_Short')['Tumor_Sample_Barcode'].nunique()
mutation_counts.sort_values(ascending=False).head(50)

HGVSp_Short
p.M1?          115
p.G12D          68
p.G12V          50
p.V600E         48
p.R175H         45
p.G13D          43
p.E545K         39
p.A2V           34
p.A146T         31
p.R282W         28
p.A2=           28
p.R248Q         27
p.A2T           27
p.R128H         27
p.R122H         27
p.R133H         26
p.R105C         26
p.R123H         25
p.A41T          24
p.R143H         24
p.R81H          23
p.R41H          23
p.R84H          23
p.R235H         23
p.A79T          23
p.R123C         23
p.A111T         22
p.X2_splice     22
p.R273H         22
p.R96C          22
p.R149H         22
p.R204Q         22
p.R151H         22
p.X5_splice     21
p.X6_splice     21
p.R115H         21
p.R124H         21
p.R138C         21
p.S117=         21
p.R128C         21
p.A54T          21
p.A3T           21
p.A212T         21
p.R161H         21
p.R169H         21
p.R139Q         21
p.R140Q         21
p.R315C         21
p.A175T         21
p.A115T         21
Name: Tumor_Sample_Barcode, dtype: int

## Classificatios using protein data

In [21]:
pathological_df = pd.read_csv(
    os.path.join(study_path, "pathological_df_v2.csv")
)
pathological_df.shape

(88, 18)

In [22]:
filtered_df.head()

,Tumor_Sample_Barcode,HGVSp_Short
0,TCGA-3L-AA1B-01,p.Y2316=
1,TCGA-3L-AA1B-01,p.V147M
2,TCGA-3L-AA1B-01,p.R1345W
3,TCGA-3L-AA1B-01,p.G254V
4,TCGA-3L-AA1B-01,p.P3615L


In [23]:
protein_df = filtered_df.rename(
    columns={"Tumor_Sample_Barcode": "SAMPLE_ID"}
)

# filter to only samples in scope
df = pd.merge(pathological_df["SAMPLE_ID"], protein_df, how="inner", on="SAMPLE_ID")
df.shape

(25006, 2)

Select which proteins to use for analysis

In [24]:
mutation_counts = df.groupby('HGVSp_Short')['SAMPLE_ID'].nunique()
mutation_counts.sort_values(ascending=False).head(10)

HGVSp_Short
p.M1?      15
p.G12D      9
p.G12V      8
p.R248Q     7
p.E545K     7
p.R165C     7
p.R122H     6
p.R175H     6
p.S117=     6
p.P311=     6
Name: SAMPLE_ID, dtype: int64

Out of 88 patients, the protein occuring in max patients occurs in only 9 patients. The data is too sparse for further analysis. Hence not taking this analysis forward. 